In [1]:
import pandas
import gripql
import itertools
import scipy.stats as stats

conn = gripql.Connection("https://bmeg.io/api", credential_file="bmeg_credentials.json")
G = conn.graph("rc5")

Find all of the samples in the [CTRP](https://portals.broadinstitute.org/ctrp/) Breast Cancer experiment

In [21]:
q = G.query().V("Program:CTRP").out("projects").out("cases").\
    has(gripql.eq("cellline_attributes.Primary Disease", "Breast Cancer")).distinct()
all_cases = []
for row in q:
    all_cases.append(row.gid)

[INFO]	2020-01-14 13:36:52,216	40 results received in 0 seconds


For the genes of interest, get Ensembl gene ids, from the HUGO symbols

In [22]:
GENES = ["PTEN", "TP53"]

In [23]:
gene_ids = {}
for i in G.query().V().hasLabel("Gene").has(gripql.within("symbol", GENES)):
    gene_ids[i.data.symbol] = i.gid

[INFO]	2020-01-14 13:36:56,003	2 results received in 0 seconds


The CTRP doesn't have direct mutation calling, but rather they used the same cell lines as the CCLE, and we can use the mutation calls from that project. So use the `same_as` edge to identify the cases from CCLE that are the same as the ones in CTRP. Then use the mutations from those samples.

In [24]:
gene_ids

{'TP53': 'ENSG00000141510', 'PTEN': 'ENSG00000171862'}

For each of the genes, find the set of samples that have a mutation in that gene

In [32]:
mut_cases = {}
norm_cases = {}

q = G.query().V(all_cases).as_("ctrp").out("same_as").has(gripql.eq("project_id", "Project:CCLE"))
q = q.out("samples").out("aliquots").out("somatic_callsets")
q = q.outE("alleles").has(gripql.within("ensembl_gene", list(gene_ids.values())))
q = q.render({"case" : "$ctrp._gid", "gene" : "$._data.ensembl_gene"})

for res in q:
    mut_cases[res.gene] = mut_cases.get(res.gene, set()) | set([res.case])

#get CCLE samples without mutation
for i in gene_ids.values():
    norm_cases[i] = list(set(all_cases).difference(mut_cases[i]))

    print( "%s Positive Set: %d" % (i, len(mut_cases[i])) )
    print( "%s Negative Set: %d" % (i, len(norm_cases[i])) )


[INFO]	2020-01-14 13:39:36,629	43 results received in 0 seconds


ENSG00000141510 Positive Set: 29
ENSG00000141510 Negative Set: 11
ENSG00000171862 Positive Set: 9
ENSG00000171862 Negative Set: 31


In [33]:
pos_response = {}
for g in gene_ids.values():
    pos_response[g] = {}
    q = G.query().V(list(mut_cases[g])).as_("a").out("samples").out("aliquots")
    q = q.out("drug_response").as_("a").out("compounds").as_("b")
    q = q.select(["a", "b"])    
    for row in q:
        v = row['a']['data']['aac']
        compound = row['b']['gid']
        if compound not in pos_response[g]:
            pos_response[g][compound] = [ v ]
        else:
            pos_response[g][compound].append(v)
   

[INFO]	2020-01-14 13:39:55,140	12,224 results received in 3 seconds
[INFO]	2020-01-14 13:39:56,594	3,999 results received in 1 seconds


In [34]:
neg_response = {}
for g in gene_ids.values():
    neg_response[g] = {}
    q = G.query().V(list(norm_cases[g])).as_("a").out("samples").out("aliquots")
    q = q.out("drug_response").as_("a").out("compounds").as_("b")
    q = q.select(["a", "b"])    
    for row in q:
        v = row['a']['data']['aac']
        compound = row['b']['gid']
        if compound not in neg_response[g]:
            neg_response[g][compound] = [ v ]
        else:
            neg_response[g][compound].append(v)
   

[INFO]	2020-01-14 13:39:59,162	5,025 results received in 1 seconds
[INFO]	2020-01-14 13:40:03,457	13,250 results received in 4 seconds


In [35]:
drugs = set(itertools.chain.from_iterable( i.keys() for i in pos_response.values() ))
out = []
for drug in drugs:
    for g in gene_ids.values():
        if drug in pos_response[g] and drug in neg_response[g]:
            row = {"drug" : drug, "mutation" : g}
            mut_values = pos_response[g][drug]
            norm_values = neg_response[g][drug]
            if len(mut_values) > 5 and len(norm_values) > 5:
                s = stats.ttest_ind(mut_values, norm_values, equal_var=False)
                row["t-statistic"] = s.statistic
                row["t-pvalue"] = s.pvalue
                s = stats.f_oneway(mut_values, norm_values)
                row["a-statistic"] = s.statistic
                row["a-pvalue"] = s.pvalue
                out.append(row)

In [36]:
pandas.DataFrame(out, columns=["drug", "mutation", "t-statistic", "t-pvalue", "a-statistic", "a-pvalue"]).sort_values("a-pvalue").head(10)

,drug,mutation,t-statistic,t-pvalue,a-statistic,a-pvalue
277,Compound:CID216345,ENSG00000141510,-4.425335,0.000366,23.811104,0.000019
593,Compound:CID16736978,ENSG00000141510,-3.177435,0.006779,12.249536,0.001356
214,Compound:CID9967941,ENSG00000141510,-3.114875,0.006419,11.046079,0.002136
31,Compound:CID9549297,ENSG00000141510,-2.573574,0.021992,10.077463,0.003071
480,Compound:CID56949517,ENSG00000171862,2.235155,0.069114,10.480699,0.003638
808,Compound:CID5472285,ENSG00000141510,-2.711139,0.015176,9.369077,0.004037
566,Compound:NO_ONTOLOGY:CID9809715,ENSG00000141510,-2.492241,0.026354,8.810857,0.005631
237,Compound:CID451668,ENSG00000171862,2.030714,0.051533,7.157550,0.008627
95,Compound:CID54686904,ENSG00000171862,2.241852,0.067998,8.668157,0.009525
656,Compound:CID11609586,ENSG00000171862,1.948182,0.066221,6.818910,0.011089
